In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import gradio as gr
import cv2

# Carrega o modelo de transferência de estilo pré-treinado
style_transfer_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def load_image(image):
    # Função para processar a imagem para o modelo
    image = image.astype(np.float32)[np.newaxis, ...] / 255.
    if image.shape[-1] == 4:
        image = image[..., :3]
    return image

def interpolate_images(baseline, target, alpha):
    return baseline + alpha * (target - baseline)

def apply_sharpness(image, intensity):
    kernel = np.array([[0, -intensity, 0],
                       [-intensity, 1 + 4 * intensity, -intensity],
                       [0, -intensity, 0]])
    sharp_image = cv2.filter2D(image, -1, kernel)
    return np.clip(sharp_image, 0, 255)

def style_transfer(content_image, style_image, style_density, content_sharpness):
    # Processa as imagens
    content_image = load_image(content_image)
    style_image = load_image(style_image)

    # Aplica nitidez na imagem de conteúdo antes da transferência de estilo
    content_image_sharp = apply_sharpness(content_image[0], intensity=content_sharpness)
    content_image_sharp = content_image_sharp[np.newaxis, ...]

    # Executa a transferência de estilo
    stylized_image = style_transfer_model(tf.constant(content_image_sharp), tf.constant(style_image))[0]

    # Interpola entre a imagem de conteúdo e a imagem estilizada para densidade de estilo
    stylized_image = interpolate_images(
        baseline=content_image[0],
        target=stylized_image.numpy(),
        alpha=style_density
    )

    # Converte a imagem resultante para o formato correto
    stylized_image = np.array(stylized_image * 255, np.uint8)

    # Remove a dimensão do batch
    stylized_image = np.squeeze(stylized_image)
    return stylized_image

iface = gr.Interface(
    fn=style_transfer,
    inputs=[
        gr.Image(label="Content Image"),  # Imagem de conteúdo
        gr.Image(label="Style Image"),    # Imagem de estilo
        gr.Slider(minimum=0, maximum=1, value=0.5, label="Adjust Style Density"),
        gr.Slider(minimum=0, maximum=1, value=0.5, label="Content Sharpness")
    ],
    outputs=gr.Image(label="Stylized Image")
)

iface.launch()
